# Instrumental variables regression

files needed = ('card.dta', )

We introduce the **linearmodels** package in this notebook [(docs)](https://bashtage.github.io/linearmodels/doc/). The linearmodels package is built on top of statsmodels and provides us with extended functionality, including instrumental variables regressions and panel regressions. 

We will use linearmodels to perform instrumental variables regressions (two stage least squares). Along the way, we will practice using ols from statsmodels a bit, too. 

We need to install linearmodels. Open a command window and type 
```python
pip install --user linearmodels
```
then hit enter. 

In [ ]:
import pandas as pd                    # for data handling
import numpy as np                     # for numerical methods and data structures
import matplotlib.pyplot as plt        # for plotting
import seaborn as sea                  # advanced plotting

import statsmodels.formula.api as smf  # provides a way to directly spec models from formulas
import linearmodels.iv as iv           # built on statsmodels, this package provides iv in an easy way 

### Medical expenditure panel survey

Let's use data from the [medical expenditure panel survey](https://meps.ahrq.gov/mepsweb/). It surveys households, medical providers, and employers. There is a lot of interesting data here. 

The linearmodels package includes a dataset based on this data. Let's load it. You can see the variable list definitions [here](https://bashtage.github.io/linearmodels/doc/iv/examples/using-formulas.html). The example below is based on the example they provide. The data cover people over the age of 65.



In [ ]:
from linearmodels.datasets import meps
med_exp = meps.load()
med_exp.head()

In [ ]:
# It's clean! 
med_exp.info()

In [ ]:
# But there are a few nans. Drop them.
med_exp = med_exp.dropna()

### OLS
We would like to understand the impact of employer (or union) provided healthcare on drug expenditure. 

$$\log(\text{drugexp}) = \beta_0 + \beta_1 \text{linc} + \beta_2 \text{hi_empunion} + \epsilon .$$

Start with ols.

In [ ]:
res_ols = smf.ols('np.log(drugexp) ~ linc + hi_empunion', data=med_exp).fit(cov_type = 'HC3')
print(res_ols.summary())

The coefficient on employer health insurance is positive. People who had insurance through work spend more on drugs in their old age...or do they?

### Instrumental variables 

The concern is that *hi_empunion* is not exogenous. We will use *ssiratio* as an instrument for *hi_emp_union*. The purpose of the instrument is to remove the endogeneity of *hi_empunion.* A good instrument is

1. Correlated with the endogenous variable (in this case, *hi_empunion*) 
2. Uncorrelated with the error term

We can test the first condition. Regress 

$$\text{hi_empunion} =  \beta_1 \text{ssiratio} + \nu .$$

In [ ]:
res_s1 = smf.ols('hi_empunion ~  ssiratio + linc', data=med_exp).fit(cov_type = 'HC3')
print(res_s1.summary())

It's significant, so it passes the the first test. The second test cannot generally be checked. 

### Two stage least squares

Note that our test of condition 1 generates fitted variables that are washed of their endogeneity. We can use the fitted variables in our original regression as an independent variable,

$$\log(\text{drugexp}) = \beta_0 + \beta_1 \text{linc} + \beta_2 \widehat{\text{hi_empunion}} + \epsilon .$$


We refer to the regression of *hi_empunion* on *ssiratio* as the **first stage regression** and the regression above as the **second stage regression**. In general, the two step procedure is

1. Regress the endogenous variable on the instrument(s)
2. Estimate the original model, after replacing the endogenous variable with the fitted values from step 1.


In [ ]:
med_exp['fitted'] = res_s1.fittedvalues
med_exp['fitted'].describe()

In [ ]:
res_s2 = smf.ols('np.log(drugexp) ~ linc + fitted', data=med_exp).fit(cov_type = 'HC3')
print(res_s2.summary())

There are two problems with this procedure. 1. It's tedious. 2. The standard errors are not correct. 

Both of these problems are solved by using a two state least squares estimator. 

### 2SLS with linearmodels

We modify the patsy notation by specifying the endogenous variable and the instrument in square brackets. A couple minor differences in syntax compared to statsmodels. 

1. We need the `.from_formula` method if we are not specifying the design matrices directly. 
2. We need to include `1` to get the constant
3. There is not a named parameter `data`. We have to pass `from_formula( formual, data)`.

In [ ]:
res_iv = iv.IV2SLS.from_formula('np.log(drugexp) ~ 1 + linc + [hi_empunion ~ ssiratio]', med_exp).fit()
print(res_iv.summary)

The point estimates are the same, but the standard errors differ. We can see the first-stage regression. Compare them to our first stage above. 

In [ ]:
print(res_iv.first_stage)

## Practice

Take a few minutes and try the following. Feel free to chat with those around you if you get stuck. The TA and I are here, too.

The file 'card.dta' contains data about individuals' wages and characteristics. The complete variable list is [here](http://fmwww.bc.edu/ec-p/data/wooldridge/card.des). 

1. Load the data. Make sure everything looks okay.

2. Scatter plot log(wage) (y-axis) and educ (x-axis). \[The log is the natural log, np.log()\]

3. Regress wages on education. Use OLS.  

$$\log(\text{wage}) = \beta_0 + \beta_1 \text{educ} + \epsilon .$$

Make life interesting. Do not use the *lwage* variable. Roll your own. 

Consider the variable *nearc4* as an instrument. The variable is equal to 1 if the person grew up near a bachelor degree granting institution. 

4. Scatter plot *educ* (y-axis) and *nearc4* (x-axis). Is this plot very useful? \[If you finish early, come back to this plot and see if you can improve on it.\]

5. Does *nearc4* satisfy the first condition for *nearc4* to be a good instrument for education?

6. Estimate  

$$\log(\text{wage}) = \beta_0 + \beta_1 \text{educ} + \epsilon .$$

using *nearc4* as an instrument for *educ*. Print out the results summary.

Make life interesting. Do not use the *lwage* variable. Roll your own. 

7. Go nuts. Estimate 

$$\log(\text{wage}) = \beta_0 + \beta_1 \text{exper} + \beta_2 \text{exper}^2 + \beta_3 \text{black} + \beta_4 \text{smsa}+ \beta_5 \text{south} + \beta_6 \text{educ} + \epsilon .$$

Instrument *educ* with both *nearc4* and *nearc2*. 

Make life interesting. Do not use the *expersq* or *lwage* variables. Roll your own. 